# Imports

In [123]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import shutil
import pandas as pd
from PIL import Image

# URL & Base links

In [136]:
base_url = "https://myanimelist.net/anime.php"
base_genre_url = "https://myanimelist.net/"

main_page_html = requests.get(base_url).content

soup = BeautifulSoup(main_page_html, "lxml")

folder = r"C:\Users\yassi\posters"
folder.replace("\\","/")

'C:/Users/yassi/posters'

# Saving the HTML

In [3]:
# Exporting the HTML to a file

def save_html(html,name):
    with open(name +".html", 'wb') as file:
        file.write(soup.prettify('utf-8'))
    
with open('main_page.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

# Getting the Links

In [4]:
# sorting_sections: We will get the sorting sections of the MAL (Genres, Explicit, Themes,....)
sorting_sections = soup.find_all('div', attrs = {'class': "normal_header pt24 mb0"})
sorting_sections

# Navigate through the child 'div' to get the links for the Genres 
genres = sorting_sections[0].find_next("div").find_all('a')
genres

# genres_links: partial links for the whole genres
genres_links = [link.get('href') for link in genres]

# raw_genres: whole list of genres
raw_genres = [genre.split('/')[-1] for genre in genres_links]

# full_genres_links: full link of the animes in each genre
full_genres_links = [urljoin(base_genre_url,link) for link in genres_links]
full_genres_links

['https://myanimelist.net/anime/genre/1/Action',
 'https://myanimelist.net/anime/genre/2/Adventure',
 'https://myanimelist.net/anime/genre/5/Avant_Garde',
 'https://myanimelist.net/anime/genre/46/Award_Winning',
 'https://myanimelist.net/anime/genre/28/Boys_Love',
 'https://myanimelist.net/anime/genre/4/Comedy',
 'https://myanimelist.net/anime/genre/8/Drama',
 'https://myanimelist.net/anime/genre/10/Fantasy',
 'https://myanimelist.net/anime/genre/26/Girls_Love',
 'https://myanimelist.net/anime/genre/47/Gourmet',
 'https://myanimelist.net/anime/genre/14/Horror',
 'https://myanimelist.net/anime/genre/7/Mystery',
 'https://myanimelist.net/anime/genre/22/Romance',
 'https://myanimelist.net/anime/genre/24/Sci-Fi',
 'https://myanimelist.net/anime/genre/36/Slice_of_Life',
 'https://myanimelist.net/anime/genre/30/Sports',
 'https://myanimelist.net/anime/genre/37/Supernatural',
 'https://myanimelist.net/anime/genre/41/Suspense']

In [5]:
# add multiple pages of the Genres
        
def all_pages_links(links):
    paged_links = []
    for link in links:
        i=1
        nxt_page= True
        while(nxt_page):
    
            new_link = f"{link}?page={i}"
            req = requests.get(new_link)
            if (int(req.status_code) == 200):
                paged_links.append(new_link)
                i+=1
            else:
                nxt_page = False
                break
    return paged_links
    
paged_full_genres_links = all_pages_links(full_genres_links)

# Saving the htmls of each page as 'html' file

""" 
for page in paged_full_genres_links:
    
    html_link = requests.get(page)
    html_name = page.split('/')[-1]
    save_html(html_link,html_name)
    """
paged_full_genres_links

['https://myanimelist.net/anime/genre/1/Action?page=1',
 'https://myanimelist.net/anime/genre/1/Action?page=2',
 'https://myanimelist.net/anime/genre/1/Action?page=3',
 'https://myanimelist.net/anime/genre/1/Action?page=4',
 'https://myanimelist.net/anime/genre/1/Action?page=5',
 'https://myanimelist.net/anime/genre/1/Action?page=6',
 'https://myanimelist.net/anime/genre/1/Action?page=7',
 'https://myanimelist.net/anime/genre/1/Action?page=8',
 'https://myanimelist.net/anime/genre/1/Action?page=9',
 'https://myanimelist.net/anime/genre/1/Action?page=10',
 'https://myanimelist.net/anime/genre/1/Action?page=11',
 'https://myanimelist.net/anime/genre/1/Action?page=12',
 'https://myanimelist.net/anime/genre/1/Action?page=13',
 'https://myanimelist.net/anime/genre/1/Action?page=14',
 'https://myanimelist.net/anime/genre/1/Action?page=15',
 'https://myanimelist.net/anime/genre/1/Action?page=16',
 'https://myanimelist.net/anime/genre/1/Action?page=17',
 'https://myanimelist.net/anime/genre/1/

# Getting the HTMLs 

In [6]:
def get_htmls(links):
    # animes_htmls_temp: list of html content of each genre link 
    animes_htmls_reqs = [requests.get(link).content for link in links]
    
    # animes_htmls: list of html  of each genre link as bs4 object
    animes_htmls = [BeautifulSoup(html, "lxml") for html in animes_htmls_reqs ]
    
    return animes_htmls

# All animes
animes_htmls = get_htmls(paged_full_genres_links)

# Scraping and Extracting the Data -  Functions

In [168]:
# (1)
def get_genre_discription(animes_htmls):
    
    # Getting the description of each genre webpage
    genres_discriptions = []
    
    for anime in animes_htmls:
        description = anime.find_all('p',attrs = {'class': "pt8 pr8 pb8 pl8 bg-color-base5"})
        genres_discriptions.append(description[0].text)
        
    # genres_discriptions: dictionary of Genre and its description
    genres_discriptions = dict(zip(raw_genres,genres_discriptions))
    
    return genres_discriptions
#(2)
def get_anime_details(anime_html):
    """ 
    Takes an anime: html bs4 file
    returns anime_name:string
            anime_id: string
            anime_link: string
            anime_html: bs4 file
    """
    # anime  header
    anime = anime_html.find_all('h2', attrs = {'class':"h2_anime_title"})[0]
   
    
    # get the link for anime
    
    try:
        link = anime.find_next("div").find('a').get('href')
        anime_link = link.replace("/video","")
        # anime name & ID
        anime_name= link.split('/')[-2]
        anime_id = link.split('/')[-3]
        
    except:
            link = None
            anime_link = None
            anime_name = None
            anime_id = None
    #print(anime.find_next("div").find('a').get('href'))
        
    

    
    
    return anime_name,anime_id,anime_link, anime

#(3)
def get_anime_temporal_details(anime):
    
    """
    Takes an anime: html bs4 file
    returns brd (Method of Broadcast):String
            year(Year of Broadcast): String
            Airing status:  String
            eps (No. Episodes): String
            run_time(Duration of episode): String
    """

    genric_data = anime.find_next('div').find_all('span', class_="item") 
    brd_year = genric_data[0].text
    brd , year = brd_year.split(',')
    brd , year = brd.strip() , year.strip()
    
    # anime status 
    status = genric_data[1].text.strip()
    
    # episodes, run_time 
    eps_run = genric_data[2].text.replace('\n',"")
    eps, run_time = eps_run.split(',')
    eps, run_time = eps.strip() , run_time.strip()
    
    return brd, year, status, eps, run_time

#(4)
def get_anime_related_genre(anime):
    
    """
    Takes an anime: html bs4 file
    returns anime_genre (Other related genre for the anime): list[String]
            anime_genre sub-tree: html bs4 file 
    """
    
    # Anime other related genres
    anime_genre_tree = anime.find_next('div').find_next('div').find_next('div').find_next('div').find_next('div')
    anime_genre = anime_genre_tree.text.strip().split('\n\n')
    
    return anime_genre,anime_genre_tree

#(5)
def get_anime_poster(anime_genre):
    
    """
    Takes an anime bs4 sub-tree (From the anime_genre tree): html bs4 file
    Saves the anime poster: jpg
    returns poster(Anime poster link): String
            anime_poster sub-tree: html bs4 file 
    """
    # Anime poster link
    anime_poster = anime_genre.find_next('div').find_next('div')
    raw = anime_poster.find_all('a')
    temp_links = str(raw).split('https')
    
    # get the poster link of a list of multiple links
    poster = temp_links[3]
    
    # cut the string from the start till the end of the word ".jpg"
    poster = "https" + poster[:poster.index(".jpg")+4]
    
    
    
    # Saving the poster as 'jpg'
    poster_pic = requests.get(poster, stream=True)
    path = f"{folder}\{anime_name}.jpg"
    path = path.replace("\\","/")
    with open(path ,'wb') as f:
        shutil.copyfileobj(poster_pic.raw, f)
    
    
    return poster,anime_poster

#(6)
def get_anime_story(anime_poster):
    """
    Takes an anime bs4 sub-tree (From the anime_poster tree): html bs4 file
    returns anime_story: String
            
    """
    # Anime story
    anime_story = anime_poster.find_next('div').find('p').text
    anime_story= anime_story.replace("\r\n\r\n","")
    anime_story= anime_story.replace("[Written by MAL Rewrite]","")
    anime_story= anime_story.replace("\r\n","")
    
    
    return anime_story

#(7)
def get_anime_studio_theme(anime_poster):
    """
    Takes an anime bs4 sub-tree (From the anime_poster tree): html bs4 file
    returns anime_studio: String
            anime_source(Manga,novel,..): String
            anime_theme: list[String]
    """
    # Anime studio, Source, Theme
    std_src_theme = anime_poster.find_next('div').find_next('div').find_all('span', class_="item")
    studio = std_src_theme[0].text
    anime_source = std_src_theme[1].text
    # Theme may contain multiple entries
    anime_theme = [ theme.text for theme in std_src_theme[2:]]
    
    return studio, anime_source, anime_theme

#(8)

def get_anime_score(anime_poster):
    
    """
    Takes an anime bs4 sub-tree (From the anime_poster tree): html bs4 file
    returns anime_score: String

    """
    
    
    score_tree = anime_poster.find_next('div').find_next('div').find_next('div').find_next('div').find_next('div').find_next('div')

    
    
    # Old : score = score_tree.find('div', class_ = "scormem-container").find_next('div').text
    # Old: score = score.replace("\n","").strip()
    #Modified:
    score = score_tree.find_next('div', class_ = "scormem-container").text
    score = score.replace("\n","").strip().split(' ')[0]
    return score

#()



# Scraping and Extracting the Data - Run

In [8]:
#(1) Getting the description of each genre webpage
genres_discriptions = get_genre_discription(animes_htmls)

In [209]:
# V2
# Extracted data list
raw_data = []
# Page number index
p=0
# Anime number index
a=0

for page in animes_htmls:
    
    animes = page.find_all('div',class_= "title-text")
    
    for anime_html in animes:
        
        #(2) Pass an anime html bs4 object and receive the main details of the anime 
        anime_name,anime_id,anime_link, anime_html_sub = get_anime_details(anime_html)
        anime_main_details = [anime_id, anime_name,anime_link]
        print(f" page({i}), Amime({a}): {anime_name}")

        ids = [row[0] for row in raw_data]

        if (anime_id in ids):
            a+=1
            continue

        #(3) Pass an anime html bs4 object and receive the Temporal details of the anime
        brd, year, status, eps, run_time = get_anime_temporal_details(anime_html_sub)
        anime_temporal_details = [brd, year, status, eps, run_time]

        #(4)
        anime_related_genres, anime_genre = get_anime_related_genre(anime_html_sub)
        all_genres = "_".join(anime_related_genres)
        #(5)
        anime_poster_link,anime_poster_tree = get_anime_poster(anime_genre)

        #(6)
        anime_story = get_anime_story(anime_poster_tree)

        #(7)
        studio, anime_source, anime_theme =  get_anime_studio_theme(anime_poster_tree)
        anime_origin = [studio, anime_source, anime_theme]
        all_themes = "_".join(anime_theme)

        #(8)
        anime_score = get_anime_score(anime_poster_tree)


        anime_all = [anime_id, anime_name, anime_score, status,
                     eps, run_time, brd, year,
                     all_genres, studio, anime_source, all_themes,
                     anime_poster_link, anime_link, anime_story]
        raw_data.append(anime_all)
        a+=1
    p+=1


 359: Shingeki_no_Kyojin
 359: Fullmetal_Alchemist__Brotherhood
 359: One_Punch_Man
 359: Sword_Art_Online
 359: Boku_no_Hero_Academia
 359: Kimetsu_no_Yaiba
 359: Naruto
 359: Tokyo_Ghoul
 359: Hunter_x_Hunter_2011
 359: Shingeki_no_Kyojin_Season_2
 359: Boku_no_Hero_Academia_2nd_Season
 359: Naruto__Shippuuden
 359: Shingeki_no_Kyojin_Season_3
 359: Jujutsu_Kaisen
 359: Code_Geass__Hangyaku_no_Lelouch
 359: One_Piece
 359: Noragami
 359: Boku_no_Hero_Academia_3rd_Season
 359: Shingeki_no_Kyojin_Season_3_Part_2
 359: Akame_ga_Kill
 359: Nanatsu_no_Taizai
 359: Mirai_Nikki_TV
 359: Mob_Psycho_100
 359: Sword_Art_Online_II
 359: Ansatsu_Kyoushitsu
 359: Ao_no_Exorcist
 359: Bleach
 359: Shingeki_no_Kyojin__The_Final_Season
 359: Kiseijuu__Sei_no_Kakuritsu
 359: Cowboy_Bebop
 359: Tokyo_Ghoul_%E2%88%9AA
 359: Neon_Genesis_Evangelion
 359: Code_Geass__Hangyaku_no_Lelouch_R2
 359: Fairy_Tail
 359: Kill_la_Kill
 359: Boku_no_Hero_Academia_4th_Season
 359: Psycho-Pass
 359: Soul_Eater
 359: 

 365: Tsuki_ga_Michibiku_Isekai_Douchuu
 365: Jormungand
 365: Kemono_Jihen
 365: Initial_D_First_Stage
 365: Berserk
 365: BNA
 365: Danganronpa_3__The_End_of_Kibougamine_Gakuen_-_Mirai-hen
 365: Sentouin_Hakenshimasu
 365: Genjitsu_Shugi_Yuusha_no_Oukoku_Saikenki
 365: Evangelion__30_10_Thrice_Upon_a_Time
 365: B__The_Beginning
 365: Suisei_no_Gargantia
 365: Phantom__Requiem_for_the_Phantom
 365: Naruto__Shippuuden_Movie_6_-_Road_to_Ninja
 365: Mahouka_Koukou_no_Rettousei__Raihousha-hen
 365: Arifureta_Shokugyou_de_Sekai_Saikyou_2nd_Season
 365: Code_Breaker
 365: K__Return_of_Kings
 365: Stranger__Mukou_Hadan
 365: Witch_Craft_Works
 365: Ben-To
 365: Tales_of_Zestiria_the_Cross
 365: Wolfs_Rain
 365: Seikon_no_Qwaser
 365: Machine-Doll_wa_Kizutsukanai
 365: Freezing
 365: Nanbaka
 365: None
 366: Fate_stay_night_Movie__Heavens_Feel_-_III_Spring_Song
 366: Seirei_Gensouki
 366: Nejimaki_Seirei_Senki__Tenkyou_no_Alderamin
 366: Fate_stay_night__Unlimited_Blade_Works_Prologue
 366: S

 380: Pokemon_Movie_03__Kesshoutou_no_Teiou_Entei
 380: Boku_no_Hero_Academia__Sukue_Kyuujo_Kunren
 380: Vampire_Hunter_D_2000
 380: Jujutsu_Kaisen_2nd_Season
 380: Heion_Sedai_no_Idaten-tachi
 380: Dragon_Crisis
 380: Berserk_2nd_Season
 380: Last_Exile
 380: Busou_Renkin
 380: Baki__Dai_Raitaisai-hen
 380: Sennen_Joyuu
 380: Trinity_Seven_Movie_1__Eternity_Library_to_Alchemic_Girl
 380: Tokyo_ESP
 380: Beatless
 380: Initial_D_Fourth_Stage
 380: Bubble
 380: Dragon_Ball_Z_Movie_15__Fukkatsu_no_F
 380: Hunter_x_Hunter_Movie_2__The_Last_Mission
 380: None
 381: None
 382: Utawarerumono
 382: Devil_Survivor_2_The_Animation
 382: Kakumeiki_Valvrave
 382: Hakuouki
 382: Kuro_no_Shoukanshi
 382: Infinite_Dendrogram
 382: Divine_Gate
 382: Nabari_no_Ou
 382: JoJo_no_Kimyou_na_Bouken_Part_6__Stone_Ocean_Part_2
 382: Kidou_Senshi_Gundam_SEED
 382: DRAMAtical_Murder
 382: Freezing_Vibration
 382: Fairy_Gone
 382: None
 383: Tensei_Kenja_no_Isekai_Life__Dai-2_no_Shokugyou_wo_Ete_Sekai_Saikyou_n

 414: Hyakka_Ryouran__Samurai_Bride
 414: None
 415: None
 416: Blood-C__The_Last_Dark
 416: One_Piece_Movie_04__Dead_End_no_Bouken
 416: Queens_Blade__Rurou_no_Senshi
 416: One_Piece_Movie_06__Omatsuri_Danshaku_to_Himitsu_no_Shima
 416: None
 417: Code_Geass__Hangyaku_no_Lelouch_I_-_Koudou
 417: Eureka_Seven_AO
 417: Kidou_Senshi_Gundam_Unicorn
 417: Mo_Dao_Zu_Shi__Xian_Yun_Pian
 417: Gantz_O
 417: Full_Metal_Panic_Invisible_Victory
 417: Muv-Luv_Alternative__Total_Eclipse
 417: Tsubasa__Tokyo_Revelations
 417: Tetsuwan_Birdy_Decode
 417: Mob_Psycho_100__Dai_Ikkai_Rei_toka_Soudansho_Ian_Ryokou_-_Kokoro_Mitasu_Iyashi_no_Tabi
 417: None
 418: Strike_the_Blood__Valkyria_no_Oukoku-hen
 418: Naruto_SD__Rock_Lee_no_Seishun_Full-Power_Ninden
 418: Noumin_Kanren_no_Skill_bakka_Agetetara_Nazeka_Tsuyoku_Natta
 418: Break_Blade
 418: FLCL_Alternative
 418: Kakumeiki_Valvrave_2nd_Season
 418: Mob_Psycho_100__Reigen_-_Shirarezaru_Kiseki_no_Reinouryokusha
 418: The_Big_O
 418: Jouran__The_Princess_

 470: Break_Blade_Movie_4__Sanka_no_Chi
 470: Senki_Zesshou_Symphogear_G
 470: Amatsuki
 470: Wizard_Barristers__Benmashi_Cecil
 470: None
 471: Peace_Maker_Kurogane
 471: Murder_Princess
 471: Hakuouki__Reimeiroku
 471: None
 472: Hypnosis_Mic__Division_Rap_Battle_-_Rhyme_Anima
 472: Senyuu_2
 472: Planet_With
 472: Hidan_no_Aria__Butei_ga_Kitarite_Onsen_Kenshuu
 472: M3__Sono_Kuroki_Hagane
 472: Digimon_Adventure_tri_4__Soushitsu
 472: Shijou_Saikyou_no_Deshi_Kenichi_OVA
 472: Fuuto_Tantei
 472: Break_Blade_Movie_5__Shisen_no_Hate
 472: Ni_no_Kuni
 472: Baraou_no_Souretsu
 472: None
 473: Dragon_Ball_Movie_3__Makafushigi_Daibouken
 473: None
 474: None
 475: Ginga_Kikoutai_Majestic_Prince
 475: Pokemon_Generations
 475: Ao_no_6-gou
 475: None
 476: None
 477: Break_Blade_Movie_6__Doukoku_no_Toride
 477: Ninja_Batman
 477: Digimon_Adventure_
 477: Persona_5_the_Animation__The_Day_Breakers
 477: Pokemon_Movie_20__Kimi_ni_Kimeta
 477: None
 478: None
 479: Queens_Blade__Gyokuza_wo_Tsugu

...

 28167: None
 28168: None
 28169: None
 28170: None
 28171: Shin_Tennis_no_Ouji-sama__Hyoutei_vs_Rikkai_-_Game_of_Future
 28171: None
 28172: None
 28173: None
 28174: None
 28175: None
 28176: None
 28177: None
 28178: None
 28179: Kyojin_no_Hoshi
 28180: None
 28181: None
 28182: None
 28183: None
 28184: None
 28185: None
 28186: None
 28187: None
 28188: Mutsugo_to_Ouma_no_Monogatari
 28189: King_of_Prism__Shiny_Seven_Stars_I_-_Prologue_x_Yukinojou_x_Taiga
 28189: None
 28190: None
 28191: None
 28192: None
 28193: None
 28194: None
 28195: None
 28196: None
 28197: None
 28198: None
 28199: None
 28200: None
 28201: None
 28202: None
 28203: None
 28204: None
 28205: None
 28206: None
 28207: None
 28208: None
 28209: None
 28210: None
 28211: None
 28212: CMFU_Xueyuan__Shenshi_Ji_Jijian
 28213: None
 28214: None
 28215: None
 28216: None
 28217: None
 28218: None
 28219: None
 28220: None
 28221: Fantasista_Stella
 28221: None
 28222: None
 28223: Uma_Musume__Pretty_Derby_-_Road_

 28665: None
 28666: Air
 28667: Mushishi_Zoku_Shou
 28668: Little_Busters
 28669: Ano_Hi_Mita_Hana_no_Namae_wo_Bokutachi_wa_Mada_Shiranai_Movie
 28670: Ajin_Part_2
 28671: Sora_no_Otoshimono__Forte
 28672: Ch%C3%A4oS_HEAd
 28673: Yuragi-sou_no_Yuuna-san
 28674: Taboo_Tattoo
 28675: Servamp
 28676: InuYasha__Kanketsu-hen
 28677: Shiguang_Dailiren
 28678: Mahoutsukai_no_Yome__Hoshi_Matsu_Hito
 28679: Noblesse
 28680: Angel_Beats_Specials
 28681: Kanon_2006
 28682: Nurarihyon_no_Mago
 28683: Fate_Grand_Order__Zettai_Majuu_Sensen_Babylonia
 28684: Higurashi_no_Naku_Koro_ni_Gou
 28685: Persona_4_the_Animation
 28686: Flying_Witch
 28687: None
 28688: Devils_Line
 28689: Mushishi_Zoku_Shou_2nd_Season
 28690: K__Missing_Kings
 28691: Kanokon
 28692: Shaman_King_2021
 28693: Persona_5_the_Animation
 28694: Bungou_Stray_Dogs__Dead_Apple
 28695: Natsume_Yuujinchou_San
 28696: None
 28697: Gankutsuou
 28698: Zoku_Natsume_Yuujinchou
 28699: Sirius
 28700: None
 28701: Omamori_Himari
 28702: Natsu

 28942: None
 28943: None
 28944: None
 28945: Persona_3_the_Movie_4__Winter_of_Rebirth
 28946: Sword_Gai_The_Animation
 28947: Bem
 28948: Jikan_no_Shihaisha
 28949: xxxHOLiC_Rou
 28950: None
 28951: None
 28952: Kai_Byoui_Ramune
 28952: Youkai_Apartment_no_Yuuga_na_Nichijou
 28953: Majin_Tantei_Nougami_Neuro
 28954: Natsume_Yuujinchou_Movie__Utsusemi_ni_Musubu
 28955: Lodoss-tou_Senki
 28956: K__Seven_Stories_Movie_1_-_R_B_-_Blaze
 28957: Aa_Megami-sama_Sorezore_no_Tsubasa
 28958: None
 28959: Durararax2_Shou__Watashi_no_Kokoro_wa_Nabe_Moyou
 28960: Wolfs_Rain_OVA
 28961: None
 28962: None
 28963: None
 28964: Summer_Ghost
 28964: Fate_Grand_Order__Zettai_Majuu_Sensen_Babylonia_-_Initium_Iter
 28965: Fate_Grand_Order__Shuukyoku_Tokuiten_-_Kani_Jikan_Shinden_Solomon
 28966: None
 28967: None
 28968: Fate_Grand_Order__Shinsei_Entaku_Ryouiki_Camelot_2_-_Paladin__Agateram
 28969: Mayonaka_no_Occult_Koumuin
 28970: Tegamibachi_Reverse
 28971: Hakkenden__Touhou_Hakken_Ibun_2nd_Season
 2897

 29323: None
 29324: None
 29325: Taishou_Mebiusline__Chicchai-san
 29325: Izumo__Takeki_Tsurugi_no_Senki
 29326: Donten_ni_Warau_Gaiden__Shukumei_Soutou_no_Fuuma
 29327: None
 29328: None
 29329: Yichang_Shengwu_Jianwenlu
 29329: Shaman_King_2021_Zoku-hen
 29330: Saiyuuki_Reload__Zeroin
 29331: None
 29332: None
 29333: None
 29334: None
 29335: None
 29336: None
 29337: None
 29338: None
 29339: Kaibutsu_Oujo__Konsui_Oujo
 29340: None
 29341: None
 29342: Doupo_Cangqiong
 29343: None
 29344: None
 29345: None
 29346: None
 29347: None
 29348: Mahoutsukai_no_Yoru
 29349: None
 29350: Furusato_Saisei__Nippon_no_Mukashibanashi
 29350: None
 29351: None
 29352: None
 29353: Yami_Shibai_9
 29354: None
 29355: Bai_Yao_Pu
 29356: None
 29357: None
 29358: None
 29359: None
 29360: None
 29361: None
 29362: Donten_ni_Warau_Gaiden__Ouka_Tenbou_no_Kakehashi
 29363: None
 29364: Bem_Movie__Become_Human
 29365: None
 29366: None
 29367: None
 29368: None
 29369: Okitegami_Kyouko_no_Bibouroku_x_M

 29849: None
 29850: Kaijuu_Sakaba_Kanpai
 29851: None
 29852: None
 29853: None
 29854: None
 29855: None
 29856: None
 29857: None
 29858: None
 29859: None
 29860: None
 29861: None
 29862: None
 29863: None
 29864: None
 29865: None
 29866: None
 29867: None
 29868: None
 29869: None
 29870: None
 29871: None
 29872: None
 29873: None
 29874: None
 29875: None
 29876: None
 29877: None
 29878: None
 29879: None
 29880: None
 29881: Shijian_Qiutu
 29882: None
 29883: None
 29884: None
 29885: None
 29886: None
 29887: None
 29888: None
 29889: Peeping_Life_x_Kaijuu_Sakaba_Kaiji__Kaijuu-tachi_ga_Iru_Tokoro
 29890: None
 29891: None
 29892: None
 29893: None
 29894: None
 29895: None
 29896: None
 29897: None
 29898: None
 29899: None
 29900: None
 29901: None
 29902: None
 29903: None
 29904: None
 29905: None
 29906: None
 29907: None
 29908: Wo_Yeye
 29908: None
 29909: None
 29910: Miki_no_Mikoto
 29911: None
 29912: None
 29913: None
 29914: None
 29915: None
 29916: None
 29917:

# Saving the data to CSV

In [210]:
# CSV headers
header_names = ['Anime ID', 'Anime Name','Score', 'Status',
               'Episodes', 'Run Time' ,'Broadcast Type', 'Year',
               'Genres', 'Studio','Source', 'Theme',
               'Poster Link' , 'Anime Link', 'Story']
df = pd.DataFrame.from_records(raw_data, columns = header_names)
df.to_csv(r'C:\Users\yassi\posters\animes.csv', sep=',')

# Draft

In [212]:
""" 

raw_data = []
i=0

for anime_html in animes_htmls:
    
        
    #(2) Pass an anime html bs4 object and receive the main details of the anime 
    anime_name,anime_id,anime_link, anime_html_sub = get_anime_details(anime_html)
    anime_main_details = [anime_id, anime_name,anime_link]
    print(f" {i}: {anime_name}")
    
    ids = [row[0] for row in raw_data]
    
    if (anime_id in ids):
        i+=1
        continue
    
    #(3) Pass an anime html bs4 object and receive the Temporal details of the anime
    brd, year, status, eps, run_time = get_anime_temporal_details(anime_html_sub)
    anime_temporal_details = [brd, year, status, eps, run_time]
    
    #(4)
    anime_related_genres, anime_genre = get_anime_related_genre(anime_html_sub)
    all_genres = "_".join(anime_related_genres)
    #(5)
    anime_poster_link,anime_poster_tree = get_anime_poster(anime_genre)
    
    #(6)
    anime_story = get_anime_story(anime_poster_tree)
    
    #(7)
    studio, anime_source, anime_theme =  get_anime_studio_theme(anime_poster_tree)
    anime_origin = [studio, anime_source, anime_theme]
    all_themes = "_".join(anime_theme)
    
    #(8)
    anime_score = get_anime_score(anime_poster_tree)
    
    
    anime_all = [anime_id, anime_name, anime_score, status,
                 eps, run_time, brd, year,
                 all_genres, studio, anime_source, all_themes,
                 anime_poster_link, anime_link, anime_story]
    raw_data.append(anime_all)
    i+=1
    
"""

    






' \n\nraw_data = []\ni=0\n\nfor anime_html in animes_htmls:\n    \n        \n    #(2) Pass an anime html bs4 object and receive the main details of the anime \n    anime_name,anime_id,anime_link, anime_html_sub = get_anime_details(anime_html)\n    anime_main_details = [anime_id, anime_name,anime_link]\n    print(f" {i}: {anime_name}")\n    \n    ids = [row[0] for row in raw_data]\n    \n    if (anime_id in ids):\n        i+=1\n        continue\n    \n    #(3) Pass an anime html bs4 object and receive the Temporal details of the anime\n    brd, year, status, eps, run_time = get_anime_temporal_details(anime_html_sub)\n    anime_temporal_details = [brd, year, status, eps, run_time]\n    \n    #(4)\n    anime_related_genres, anime_genre = get_anime_related_genre(anime_html_sub)\n    all_genres = "_".join(anime_related_genres)\n    #(5)\n    anime_poster_link,anime_poster_tree = get_anime_poster(anime_genre)\n    \n    #(6)\n    anime_story = get_anime_story(anime_poster_tree)\n    \n    #(